In [43]:
import cv2
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt

In [57]:
class Armor:
    def __init__(self):
        pass
    
    def preprocess(self, img):
        self._img = img
        self._gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    def color_thresh(self, color):
        bgr = cv2.split(self._img)
        result = None
        if (color == 'red'):
            result = bgr[2] - bgr[1]
            thresh = 50
        elif (color == 'blue'):
            result = bgr[0] - bgr[2]
            thresh = 90
        if result is None:
            return None
        ret, result = cv2.threshold(result, thresh, 255, cv2.THRESH_BINARY)
        if not ret:
            return None
        kernal = np.ones((3,3), dtype=np.uint8)
        result = cv2.dilate(result, kernal, iterations=1)
        return result

    def gray_thresh(self):
        ret, result = cv2.threshold(self._gray_img, 200, 255, cv2.THRESH_BINARY)
        if not ret:
            return None
        else:
            return result

    def light_detect(self, color):
        light_bin = self.color_thresh(color)
        gray_bin = self.gray_thresh()
        
        cnt_method = cv2.CHAIN_APPROX_SIMPLE
        cnt_mode = cv2.RETR_EXTERNAL
        im, light_cnt, hier = cv2.findContours(light_bin, cnt_method, cnt_mode)
        im, gray_cnt, hier = cv2.findContours(gray_bin, cnt_method, cnt_mode)
        
        proc_flag = np.zeros()

In [58]:
armor = Armor()

In [59]:
cap = cv2.VideoCapture('./test.MOV')
while True:
    ret, img = cap.read()
    img = cv2.resize(img, (480,320))
    armor.preprocess(img)
    color_mask = armor.color_thresh('red')
    gray_mask = armor.gray_thresh()
    color_mask = color_mask & gray_mask
    cv2.imshow('img', img)
    cv2.imshow('color', color_mask)
    cv2.imshow('gray', gray_mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

error: /tmp/opencv-20180102-35761-1ch8vve/opencv-3.4.0/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
